In [14]:
# make sure ultralytics, cv2 are installed


# imports
import cv2
from ultralytics import YOLO
from collections import defaultdict
import numpy as np

In [15]:
# pretrained model
model = YOLO("yolov8n.pt")

In [16]:
# open video file
video_1 = "cars_1.mp4"
cap = cv2.VideoCapture(video_1)
# Define the codec and create VideoWriter object
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
out = cv2.VideoWriter('output_video.avi', cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), 30, (frame_width, frame_height))
# Store the track history
track_history = defaultdict(lambda: [])

In [17]:
# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLOv8 tracking on the frame, persisting tracks between frames
        results = model.track(frame, persist=True)

        # Get the boxes and track IDs
        boxes = results[0].boxes.xywh.cpu()
        track_ids = results[0].boxes.id.int().cpu().tolist()

        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        # Plot the tracks
        for box, track_id in zip(boxes, track_ids):
            x, y, w, h = box
            track = track_history[track_id]
            track.append((float(x), float(y)))  # x, y center point
            if len(track) > 30:  # retain 90 tracks for 90 frames
                track.pop(0)

            # Draw the tracking lines
            points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
            cv2.polylines(annotated_frame, [points], isClosed=False, color=(230, 230, 230), thickness=10)
            
        # Write the annotated frame to the output video
        out.write(annotated_frame)
        # Display the annotated frame
        cv2.imshow("YOLOv8 Tracking", annotated_frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()

0: 384x640 12 cars, 1 truck, 99.8ms
Speed: 3.2ms preprocess, 99.8ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)
0: 384x640 12 cars, 1 truck, 167.1ms
Speed: 3.6ms preprocess, 167.1ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)
0: 384x640 13 cars, 1 truck, 105.4ms
Speed: 2.5ms preprocess, 105.4ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)
0: 384x640 12 cars, 1 bus, 1 truck, 120.3ms
Speed: 3.1ms preprocess, 120.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
0: 384x640 12 cars, 2 trucks, 98.4ms
Speed: 2.8ms preprocess, 98.4ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)
0: 384x640 13 cars, 1 bus, 1 truck, 80.1ms
Speed: 2.6ms preprocess, 80.1ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)
0: 384x640 12 cars, 2 trucks, 78.1ms
Speed: 2.3ms preprocess, 78.1ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)
0: 384x640 12 cars, 2 trucks, 84.6ms
Speed: 2.2